In [ ]:
# --- Environment Setup ---
import sys
print("Python executable:", sys.executable)

# Install required packages into THIS environment
!{sys.executable} -m pip install numpy scipy matplotlib pandas

# Hybrid Digital Twin - Pharma Style

This notebook demonstrates a hybrid digital twin framework combining mechanistic models with machine learning corrections for pharmaceutical process simulation.

# Governing Equations and Modeling Assumptions

## Process Description
This project models a non-isothermal batch reactor carrying out consecutive first-order liquid-phase reactions:

A → B → C

The objective is to maximize the concentration of intermediate B by controlling the reactor temperature profile and batch time.

This formulation is representative of batch pharmaceutical and specialty chemical manufacturing processes where temperature history strongly affects yield and impurity formation.

The mechanistic model follows standard formulations commonly presented in process dynamics and control literature.

## Assumptions
* Perfect mixing in the reactor
* Constant reactor volume
* Constant density and heat capacity
* Liquid-phase reactions
* No inflow or outflow during the batch
* Lumped heat transfer model
* Jacket temperature is treated as a known input

## Reaction Kinetics
The reactions are first-order with Arrhenius temperature dependence:

$$k_1(T) = k_{0,1} \exp\left(\frac{-E_1}{RT}\right)$$

$$k_2(T) = k_{0,2} \exp\left(\frac{-E_2}{RT}\right)$$

**Where:**
- $k_1, k_2$ = reaction rate constants
- $k_{0,1}, k_{0,2}$ = pre-exponential factors
- $E_1, E_2$ = activation energies
- $R$ = universal gas constant
- $T$ = reactor temperature

## Mass Balances

### Component A:
$$\frac{dC_A}{dt} = -k_1(T) C_A$$

### Component B:
$$\frac{dC_B}{dt} = k_1(T) C_A - k_2(T) C_B$$

### Component C:
$$\frac{dC_C}{dt} = k_2(T) C_B$$

**Where:**
- $C_A, C_B, C_C$ = concentrations of components A, B, and C
- $t$ = time

## Energy Balance

The energy balance for the reacting mass is:

$$\rho C_p \frac{dT}{dt} = -\Delta H_1 k_1(T) C_A - \Delta H_2 k_2(T) C_B + \frac{UA}{V}(T_j - T)$$

**Where:**
- $\rho$ = density
- $C_p$ = heat capacity
- $\Delta H_1, \Delta H_2$ = heats of reaction
- $U$ = overall heat transfer coefficient
- $A$ = heat transfer area
- $V$ = reactor volume
- $T_j$ = jacket temperature
- $T$ = reactor temperature

## Notes on Extension

This model can be extended to include:
- Explicit jacket energy balance
- Temperature controller dynamics
- Measurement noise and disturbances
- Hybrid data-driven correction terms

These extensions are consistent with digital twin and advanced process
modeling workflows.



In [ ]:
# Import libraries
from __future__ import annotations
from dataclasses import dataclass
from typing import Callable, Dict, Tuple
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Load data
data = pd.read_csv('../data/simulated_batch_data.csv')
data.head()

# Implement mechanistic model

In [ ]:
# Import models
import sys
sys.path.append('..')

from model.mechanistic_model import MechanisticModel, Params, default_Tj_profile

params = Params(
    k01=8.0,
    E1=10000,
    k02=2.0,
    E2=15000,
    dH1=-45000,
    dH2=-35000,
    rhoCp=4200,
    UA_over_V=200
)

Tj = default_Tj_profile(330.0)

model = MechanisticModel(params, Tj)

CA0 = data["concentration"].iloc[0]
CB0 = 0.0
CC0 = 0.0
T0  = data["temperature"].iloc[0] + 273.15  # convert °C → K if needed

y0 = np.array([CA0, CB0, CC0, T0])

t_eval = data["time"].values

results = model.simulate(
    (t_eval.min(), t_eval.max()),
    y0,
    t_eval=t_eval
)

import matplotlib.pyplot as plt

plt.figure(figsize=(10,4))
plt.plot(results["t"], results["CA"], label="Model CA")
plt.scatter(data["time"], data["concentration"], color="red", label="Measured")
plt.xlabel("Time")
plt.ylabel("Concentration")
plt.legend()
plt.title("Mechanistic Model vs Measured Data")
plt.show()

## Next Steps
1. Train hybrid correction layer
2. Validate combined model